In [1]:
import pandas as pd
from datetime import datetime, date, timedelta, time
import pickle
import warnings

warnings.filterwarnings('ignore')

At the DOODL Supermarket chain we would like to understand our customers better in order to optimize the layout, staffing and service of our
supermarkets. We would like you and your team to model the way customers move through a representative shop. Our main business goals are:
* understand customer behavior
* explain customer behavior to our non-data staff
* optimize staffing so that the queues do not get unnecessary long

The customers can move between these areas freely. Sooner or later, they will enter the checkout area. Once they do, they are considered to have
left the shop.

In [2]:
monday = pd.read_csv("./data/monday.csv", delimiter=";", parse_dates=['timestamp'])

In [3]:
monday.head()

,timestamp,customer_no,location
0,2019-09-02 07:03:00,1,dairy
1,2019-09-02 07:03:00,2,dairy
2,2019-09-02 07:04:00,3,dairy
3,2019-09-02 07:04:00,4,dairy
4,2019-09-02 07:04:00,5,spices


In [4]:
monday.tail()

,timestamp,customer_no,location
4879,2019-09-02 21:49:00,1442,checkout
4880,2019-09-02 21:49:00,1444,checkout
4881,2019-09-02 21:49:00,1445,dairy
4882,2019-09-02 21:50:00,1446,dairy
4883,2019-09-02 21:50:00,1447,fruit


In [5]:
monday.shape

(4884, 3)

In [6]:
monday["customer_no"].nunique()

1447

**Observation:**
- On monday, there were 1447 customers in the shop

In [7]:
monday["location"].value_counts()

checkout    1437
fruit       1005
dairy        895
drinks       797
spices       750
Name: location, dtype: int64

**Observation**:
- Apparently only 1437 customers were registered at the checkout - how can the difference of 10 customers be explained?

In [8]:
mask = monday["location"] == "checkout"
checkout_customers = monday[mask]["customer_no"].tolist()
no_checkout_customers = list()

for i in range(1, monday["customer_no"].nunique() + 1):
    if i not in checkout_customers:
        no_checkout_customers.append(i)

no_checkout_customers

[1430, 1433, 1437, 1439, 1440, 1441, 1443, 1445, 1446, 1447]

**Observation**:
- These ten customers were not registered at the checkout on Monday
- They arrived all pretty late, so that their checkout couldn't be registered
- Approach: Add a new line for all these customers with a checkout at 21:55

In [9]:
def main(list, file_path):

    full_dataframe = 0
    no_checkout = {}

    counter = 0

    for day in list:
        path = file_path + day + ".csv"
        prefix = day[:3] + "_"
        df = pd.read_csv(path, delimiter=";", parse_dates=['timestamp'])

        # Resample the data in order to get an observation for every minute
        df = df.set_index('timestamp').groupby('customer_no').apply(resample_customer)
        print(df)
        df = df.drop("customer_no", axis=1).reset_index()

        # Get the customer numbers of customers that were not registered checking out
        no_checkout_customers = get_no_checkout_customer_numbers(df)

        # Adding the customer_no that were not registered at checkout to the dict
        no_checkout[prefix] = no_checkout_customers

        # Add rows with checkout time to the dataframe
        df = add_row_with_checkout_time(df, no_checkout_customers)

        # Add a new column with the shopping duration in seconds
        df = df.groupby('customer_no').apply(add_shopping_duration)

        # Resample the data in order to get an observation for every minute
        # df = df.groupby('customer_no').apply(resample_customer)

        # Add a 'after'-column
        df = df.groupby('customer_no').apply(add_after_column)

        # Add 'first' or 'following'
        df = df.groupby('customer_no').apply(first_or_following)

        # Add a prefix to the customer_no
        df = add_prefix(df, prefix)

        # Add a column with the extracted hour
        df = add_hour(df)

        counter += 1

        if counter == 1:
            full_dataframe = df
        else:
            full_dataframe = pd.concat([full_dataframe, df], ignore_index=True)

    # Rename column 'location' to 'before'
    full_dataframe = full_dataframe.rename(columns={"location": "before"})

    #full_dataframe['after'] = full_dataframe['after'].fillna("checkout")

    return full_dataframe, no_checkout

def resample_customer(group):
    group = group.resample('1min').ffill()

    return group

def get_no_checkout_customer_numbers(df):
    mask = df["location"] == "checkout"
    checkout_customers = df[mask]["customer_no"].tolist()
    no_checkout_customers = list()

    for i in range(1, df["customer_no"].nunique() + 1):
        if i not in checkout_customers:
            no_checkout_customers.append(i)

    return no_checkout_customers


def add_row_with_checkout_time(df, no_checkout):
    #Take the last timestamp of the dataframe and add five minutes
    date = df["timestamp"].iloc[-1] + timedelta(minutes=5)
    location = "checkout"

    for i in no_checkout:
        new_row = pd.Series({'timestamp': date, 'customer_no':i, 'location': location})
        #append row to the dataframe
        df = df.append(new_row, ignore_index=True)

    return df

def add_shopping_duration(group):

    # set all cost values to the mean
    group['shopping_duration'] = (group["timestamp"].max() - group["timestamp"].min()).seconds

    return group

def add_after_column(group):

    # set all cost values to the mean
    group['after'] = group["location"].shift(-1)

    return group

def first_or_following(group):
    first_row = group.iloc[:1,:]
    next_rows = group.iloc[1:,:]

    first_row["first_or_following"] = "first"
    next_rows["first_or_following"] = "following"

    group = pd.concat([first_row, next_rows])
    return group

def add_prefix(df, prefix):
    prefix = prefix
    df['customer_no'] = prefix + df['customer_no'].astype(str)

    return df

def add_hour(df):
    df["hour"] = df["timestamp"].dt.hour

    return df


weekdays = ["monday", "tuesday", "wednesday", "thursday", "friday"]

df, no_checkout = main(weekdays, "./data/")

                                 customer_no  location
customer_no timestamp                                 
1           2019-09-02 07:03:00            1     dairy
            2019-09-02 07:04:00            1     dairy
            2019-09-02 07:05:00            1  checkout
2           2019-09-02 07:03:00            2     dairy
            2019-09-02 07:04:00            2     dairy
...                                      ...       ...
1444        2019-09-02 21:48:00         1444    spices
            2019-09-02 21:49:00         1444  checkout
1445        2019-09-02 21:49:00         1445     dairy
1446        2019-09-02 21:50:00         1446     dairy
1447        2019-09-02 21:50:00         1447     fruit

[10765 rows x 2 columns]
                                 customer_no  location
customer_no timestamp                                 
1           2019-09-03 07:02:00            1     fruit
            2019-09-03 07:03:00            1     fruit
            2019-09-03 07:04:00        

In [10]:
df.shape

(53931, 7)

In [11]:
df[df["customer_no"] == "mon_6"]

,customer_no,timestamp,before,shopping_duration,after,first_or_following,hour
17,mon_6,2019-09-02 07:04:00,spices,1500,dairy,first,7
18,mon_6,2019-09-02 07:05:00,dairy,1500,dairy,following,7
19,mon_6,2019-09-02 07:06:00,dairy,1500,dairy,following,7
20,mon_6,2019-09-02 07:07:00,dairy,1500,dairy,following,7
21,mon_6,2019-09-02 07:08:00,dairy,1500,dairy,following,7
22,mon_6,2019-09-02 07:09:00,dairy,1500,dairy,following,7
23,mon_6,2019-09-02 07:10:00,dairy,1500,dairy,following,7
24,mon_6,2019-09-02 07:11:00,dairy,1500,dairy,following,7
25,mon_6,2019-09-02 07:12:00,dairy,1500,dairy,following,7
26,mon_6,2019-09-02 07:13:00,dairy,1500,dairy,following,7


In [12]:
def no_transformation(list, file_path):

    full_dataframe_no_transition = 0
    counter = 0

    for day in list:
        path = file_path + day + ".csv"
        prefix = day[:3] + "_"
        df = pd.read_csv(path, delimiter=";", parse_dates=['timestamp'])

        # Add a prefix to the customer_no
        df = add_prefix(df, prefix)

        counter += 1

        if counter == 1:
            full_dataframe_no_transition = df
        else:
            full_dataframe_no_transition = pd.concat([full_dataframe_no_transition, df], ignore_index=True)

    return full_dataframe_no_transition

def add_prefix(df, prefix):
    prefix = prefix
    df['customer_no'] = prefix + df['customer_no'].astype(str)

    return df

weekdays = ["monday", "tuesday", "wednesday", "thursday", "friday"]

df_no_transition = no_transformation(weekdays, "./data/")

In [13]:
with open('./data/full_df.bin', 'wb') as f:
    pickle.dump(df, f)

In [14]:
with open('./data/no_checkout.bin', 'wb') as f:
    pickle.dump(no_checkout, f)

In [15]:
with open('./data/df_no_transition.bin', 'wb') as f:
    pickle.dump(df_no_transition, f)